In [1]:
import pandas as pd

In [2]:
# import the files as dataframes

location_list = pd.read_csv('LMFull.csv')
category_list = pd.read_csv('tweet_categories.csv')
tweet_list = pd.read_csv('TweetTest_Unmatched.csv')

In [3]:
# This function goes through each entry in location_list and looks for it in the passed string
# When it finds a match, it returns the associated lat-long as a dict
# It ignores case. If there are multiple matches only the last one will be returned

def check_landmark(test_string):
    result = {'lat':'NONE','lon':'NONE'}
    for landmark in location_list.itertuples():
        if str(landmark[1]).lower() in test_string.lower():
            result = {'lat':landmark[4],'lon':landmark[5]}
    return result

In [4]:
# This function replaces any missing lat-long values, if check_landmark returns a match
# 'NONE' is filled in if no location match was found

def fill_in_locations(df, lat_col=3, lon_col=4):
    for row in df.itertuples():
        idx = row[0]
        location = check_landmark(str(row[3]))
        if (row[4]==0 or row[5]==0):
            df.iloc[idx,lat_col] = location['lat']
            df.iloc[idx,lon_col] = location['lon']

In [5]:
# This function goes through each entry in category_list and looks for it in the passed string
# When it finds a match, it returns the associated category name
# It ignores case. If there are multiple matches only the last one will be returned

def check_category(test_string):
    result = 'NONE'
    for keyword in category_list.itertuples():
        if str(keyword[1]).lower() in test_string.lower():
            result = keyword[2]
    return result

In [6]:
# This function replace any missing categories, if check_category returns a match

def fill_in_categories(df, cat_col=5):
    for row in df.itertuples():
        idx = row[0]
        category = check_category(str(row[3]))
        if row[6]=='NONE':
            df.iloc[idx,cat_col] = category

In [7]:
# Now run both functions to replace everything

fill_in_categories(tweet_list)
fill_in_locations(tweet_list)

In [8]:
# Add a column required by the database
tweet_list['geom'] = ""
for row in tweet_list.itertuples():
    idx = row[0]
    tweet_list.iloc[idx,6] = "SRID=4326;POINT(" + str(row[4]) + " " + str(row[5]) + ")"
# tweet_list[:10]

In [9]:
#Output to CSV

tweet_list.to_csv('matched_tweets.csv', index=False)

In [ ]:
# Check to see how many we fixed

m=0
n=0
for row in tweet_list.itertuples():
    if row[4]=='NONE' or row[5]=='NONE':
        m+=1
    else:
        n+=1

print str(n)+" have lat-long data, "+str(m)+" don't"

In [ ]:
i=0
j=0
for row in tweet_list.itertuples():
    if row[6]=='NONE':
        i+=1
    else:
        j+=1
print str(j)+" have a category, "+str(i)+" do not"